In [1]:
import pandas as pd

columns = [
        'join_id','dt','chain'
        ,'num_l1_txs_inbox','num_l1_txs_output','calldata_bytes_l1_inbox'
        ,'avg_l1_gas_price_on_l1_inbox','avg_l1_gas_price_on_l1_output','avg_l1_gas_price_on_l2'
        ,'l1_gas_used_inbox','l1_gas_used_output','l1_gas_used_combined'
        ,'l1_eth_fees_inbox','l1_eth_fees_output','l1_eth_fees_combined'
        ,'l1_contrib_l2_eth_fees_per_day','l2_contrib_l2_eth_fees_per_day'
        ,'l2_num_txs_per_day','l1_gas_used_on_l2','calldata_bytes_l2_per_day','l2_gas_used','l2_eth_fees_per_day'
        ]

In [2]:
#Unify datasets
dunedf = pd.read_csv('csv_inputs/op_chain_gas_economics_dune_query_2453515.csv')
goldskydf = pd.read_csv('csv_inputs/op_chain_gas_economics_goldsky_zora_pgn.csv')

In [3]:
#Configure
#rename cols
dunedf = dunedf.rename(columns={
                        'name':'chain'
                        })
#Generate Join ID Column
dunedf['join_id'] = dunedf['dt'].astype(str).str[:10].str.cat(dunedf['chain'].astype(str).str.lower())
goldskydf['join_id'] = goldskydf['dt'].astype(str).str[:10].str.cat(goldskydf['chain'].astype(str).str.lower())

# display(dunedf.head(10))
# print(dunedf.columns)
# print('---')
# display(goldskydf.head(10))
# print(goldskydf.columns)

combo_df = dunedf.merge(goldskydf, on='join_id', how='left')

# display(combo_df)

for c in columns:
        try:
                combo_df[c] = combo_df[c+'_x'].combine_first(combo_df[c+'_y']) #pick first non-null
        except:
                combo_df[c] = combo_df[c] #nada

combo_df['dt'] = pd.to_datetime(combo_df['dt'])
combo_df = combo_df[columns]

KeyError: 'avg_l1_gas_price'

In [ ]:
display(combo_df)

combo_df.to_csv('outputs/op_chain_gas_economics_sample.csv')